## Images Exercise 1 

In [4]:
import numpy as np

def add_noise_to_image(image):
    mean = 0
    std_dev = 25  # standard deviation

    noise = np.random.normal(mean, std_dev, image.shape).astype(np.int16)
    noisy_image = image.astype(np.int16) + noise

    # Clip values to [0, 255] and convert back to uint8
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    return noisy_image

In [8]:
import numpy as np
from PIL import Image, ImageDraw
import os
import random
import math

os.makedirs('images', exist_ok=True)
os.makedirs('groundTruth', exist_ok=True)

width, height = 256, 256

for img_num in range(50):
    # generate base image with objects (original code)
    base_r = random.randint(0, 255)
    base_g = random.randint(0, 255)
    base_b = random.randint(0, 255)
    img_array = np.zeros((height, width, 3), dtype=np.uint8)
    # img_array[:, :] = [base_r, base_g, base_b] #If we want background with colors
    img_array= add_noise_to_image(img_array)
    gt_array = Image.new("RGB", (width, height), (0, 0, 0)) 
    
    num_objects = random.randint(1, 5)
    for _ in range(num_objects):
        base_r = random.randint(0, 255)
        base_g = random.randint(0, 255)
        base_b = random.randint(0, 255)
        contour_base = (max(base_r - 50, 0), max(base_g - 50, 0), max(base_b - 50, 0))
        interior_base = (min(base_r + 50, 255), min(base_g + 50, 255), min(base_b + 50, 255))
        
        x_center = random.randint(0, width - 1)
        y_center = random.randint(0, height - 1)
        shape_type = random.choice(['circle', 'ellipse', 'rectangle', 'polygon'])
        
        mask = Image.new('L', (width, height), 0)
        draw = ImageDraw.Draw(mask)
        
        if shape_type == 'circle':
            radius = random.randint(10, 50)
            left = max(x_center - radius, 0)
            top = max(y_center - radius, 0)
            right = min(x_center + radius, width - 1)
            bottom = min(y_center + radius, height - 1)
            draw.ellipse((left, top, right, bottom), fill=1)
        elif shape_type == 'ellipse':
            rx = random.randint(10, 50)
            ry = random.randint(10, 50)
            left = max(x_center - rx, 0)
            top = max(y_center - ry, 0)
            right = min(x_center + rx, width - 1)
            bottom = min(y_center + ry, height - 1)
            draw.ellipse((left, top, right, bottom), fill=1)
        elif shape_type == 'rectangle':
            w = random.randint(10, 100)
            h = random.randint(10, 100)
            left = max(x_center - w // 2, 0)
            top = max(y_center - h // 2, 0)
            right = min(x_center + w // 2, width - 1)
            bottom = min(y_center + h // 2, height - 1)
            draw.rectangle((left, top, right, bottom), fill=1)
        elif shape_type == 'polygon':
            num_points = random.randint(3, 6)
            points = []
            for _ in range(num_points):
                dx = random.randint(-50, 50)
                dy = random.randint(-50, 50)
                px = max(0, min(x_center + dx, width - 1))
                py = max(0, min(y_center + dy, height - 1))
                points.append((px, py))
            draw.polygon(points, fill=1)
        
        mask_np = np.array(mask, dtype=bool)
        padded = np.pad(mask_np, ((1, 1), (1, 1)), constant_values=0)
        left_neighbor = padded[1:-1, :-2] == 0
        right_neighbor = padded[1:-1, 2:] == 0
        top_neighbor = padded[:-2, 1:-1] == 0
        bottom_neighbor = padded[2:, 1:-1] == 0
        contour = (left_neighbor | right_neighbor | top_neighbor | bottom_neighbor) & mask_np
        contour_img = Image.fromarray(contour)
        # Add shape to ground-truth
        gt_array.paste(contour_img, (0, 0), contour_img)
        interior = mask_np & ~contour
        
        noise_r = np.random.randint(-20, 21, (height, width))
        contour_r = np.clip(contour_base[0] + noise_r, 0, 255).astype(np.uint8)
        img_array[contour, 0] = contour_r[contour]
        noise_g = np.random.randint(-20, 21, (height, width))
        contour_g = np.clip(contour_base[1] + noise_g, 0, 255).astype(np.uint8)
        img_array[contour, 1] = contour_g[contour]
        noise_b = np.random.randint(-20, 21, (height, width))
        contour_b = np.clip(contour_base[2] + noise_b, 0, 255).astype(np.uint8)
        img_array[contour, 2] = contour_b[contour]
        
        noise_r = np.random.randint(-20, 21, (height, width))
        interior_r = np.clip(interior_base[0] + noise_r, 0, 255).astype(np.uint8)
        img_array[interior, 0] = interior_r[interior]
        noise_g = np.random.randint(-20, 21, (height, width))
        interior_g = np.clip(interior_base[1] + noise_g, 0, 255).astype(np.uint8)
        img_array[interior, 1] = interior_g[interior]
        noise_b = np.random.randint(-20, 21, (height, width))
        interior_b = np.clip(interior_base[2] + noise_b, 0, 255).astype(np.uint8)
        img_array[interior, 2] = interior_b[interior]

    # Save ground-truth image
    gt_array.save(f'groundTruth/image_{img_num:05d}.png')
    
    # composite with base image
    base_img = Image.fromarray(img_array).convert('RGBA')
    
    # save
    base_img.save(f'images/image_{img_num:05d}.png')

## Images Exercise 2

In [7]:
import numpy as np
from PIL import Image, ImageDraw
import os
import random
import math

os.makedirs('images2', exist_ok=True)
os.makedirs('groundTruth2', exist_ok=True)

width, height = 256, 256

for img_num in range(50):
    # generate base image with objects (original code)
    img_array = np.zeros((height, width, 3), dtype=np.uint8)
    img_array = add_noise_to_image(img_array)
    gt_array = Image.new("RGB", (width, height), (0, 0, 0)) 
    
    num_objects = random.randint(1, 5)
    for _ in range(num_objects):
        base_r = random.randint(0, 255)
        base_g = random.randint(0, 255)
        base_b = random.randint(0, 255)
        contour_base = (max(base_r - 50, 0), max(base_g - 50, 0), max(base_b - 50, 0))
        interior_base = (min(base_r + 50, 255), min(base_g + 50, 255), min(base_b + 50, 255))
        
        x_center = random.randint(0, width - 1)
        y_center = random.randint(0, height - 1)
        shape_type = random.choice(['circle', 'ellipse', 'rectangle', 'polygon'])
        
        mask = Image.new('L', (width, height), 0)
        draw = ImageDraw.Draw(mask)
        
        if shape_type == 'circle':
            radius = random.randint(10, 50)
            left = max(x_center - radius, 0)
            top = max(y_center - radius, 0)
            right = min(x_center + radius, width - 1)
            bottom = min(y_center + radius, height - 1)
            draw.ellipse((left, top, right, bottom), fill=1)
        elif shape_type == 'ellipse':
            rx = random.randint(10, 50)
            ry = random.randint(10, 50)
            left = max(x_center - rx, 0)
            top = max(y_center - ry, 0)
            right = min(x_center + rx, width - 1)
            bottom = min(y_center + ry, height - 1)
            draw.ellipse((left, top, right, bottom), fill=1)
        elif shape_type == 'rectangle':
            w = random.randint(10, 100)
            h = random.randint(10, 100)
            left = max(x_center - w // 2, 0)
            top = max(y_center - h // 2, 0)
            right = min(x_center + w // 2, width - 1)
            bottom = min(y_center + h // 2, height - 1)
            draw.rectangle((left, top, right, bottom), fill=1)
        elif shape_type == 'polygon':
            num_points = random.randint(3, 6)
            points = []
            for _ in range(num_points):
                dx = random.randint(-50, 50)
                dy = random.randint(-50, 50)
                px = max(0, min(x_center + dx, width - 1))
                py = max(0, min(y_center + dy, height - 1))
                points.append((px, py))
            draw.polygon(points, fill=1)
        
        mask_np = np.array(mask, dtype=bool)
        padded = np.pad(mask_np, ((1, 1), (1, 1)), constant_values=0)
        left_neighbor = padded[1:-1, :-2] == 0
        right_neighbor = padded[1:-1, 2:] == 0
        top_neighbor = padded[:-2, 1:-1] == 0
        bottom_neighbor = padded[2:, 1:-1] == 0
        contour = (left_neighbor | right_neighbor | top_neighbor | bottom_neighbor) & mask_np
        contour_img = Image.fromarray(contour)
        # Add shape to ground-truth
        gt_array.paste(contour_img, (0, 0), contour_img)
        interior = mask_np & ~contour
        
        noise_r = np.random.randint(-20, 21, (height, width))
        contour_r = np.clip(contour_base[0] + noise_r, 0, 255).astype(np.uint8)
        img_array[contour, 0] = contour_r[contour]
        noise_g = np.random.randint(-20, 21, (height, width))
        contour_g = np.clip(contour_base[1] + noise_g, 0, 255).astype(np.uint8)
        img_array[contour, 1] = contour_g[contour]
        noise_b = np.random.randint(-20, 21, (height, width))
        contour_b = np.clip(contour_base[2] + noise_b, 0, 255).astype(np.uint8)
        img_array[contour, 2] = contour_b[contour]
        
        noise_r = np.random.randint(-20, 21, (height, width))
        interior_r = np.clip(interior_base[0] + noise_r, 0, 255).astype(np.uint8)
        img_array[interior, 0] = interior_r[interior]
        noise_g = np.random.randint(-20, 21, (height, width))
        interior_g = np.clip(interior_base[1] + noise_g, 0, 255).astype(np.uint8)
        img_array[interior, 1] = interior_g[interior]
        noise_b = np.random.randint(-20, 21, (height, width))
        interior_b = np.clip(interior_base[2] + noise_b, 0, 255).astype(np.uint8)
        img_array[interior, 2] = interior_b[interior]

    # Save ground-truth image
    gt_array.save(f'groundTruth2/image_{img_num:05d}.png')

    # add semi-transparent stripes
    overlay = Image.new('RGBA', (width, height), (0, 0, 0, 0))
    draw_overlay = ImageDraw.Draw(overlay)
    
    num_stripes = random.randint(5, 15)
    for _ in range(num_stripes):
        # Random start point
        x1 = random.randint(0, width)
        y1 = random.randint(0, height)
        # Random angle and length
        angle = math.radians(random.uniform(0, 180))
        length = random.randint(50, 200)
        x2 = x1 + int(length * math.cos(angle))
        y2 = y1 + int(length * math.sin(angle))
        x2 = max(0, min(x2, width - 1))
        y2 = max(0, min(y2, height - 1))
        
        # random thickness and base color
        thickness = random.randint(1, 5)
        base_r = random.randint(0, 255)
        base_g = random.randint(0, 255)
        base_b = random.randint(0, 255)
        draw_overlay.line([(x1, y1), (x2, y2)], fill=(base_r, base_g, base_b, 255), width=thickness)
    
    # convert overlay to numpy array for processing
    overlay_np = np.array(overlay)
    # Find all stripe pixels (where alpha > 0)
    stripe_mask = overlay_np[:, :, 3] > 0
    
    # add RGB noise only to stripe pixels
    noise_r = np.random.randint(-30, 31, size=overlay_np.shape[:2])
    noise_g = np.random.randint(-30, 31, size=overlay_np.shape[:2])
    noise_b = np.random.randint(-30, 31, size=overlay_np.shape[:2])
    
    overlay_np[stripe_mask, 0] = np.clip(overlay_np[stripe_mask, 0] + noise_r[stripe_mask], 0, 255)
    overlay_np[stripe_mask, 1] = np.clip(overlay_np[stripe_mask, 1] + noise_g[stripe_mask], 0, 255)
    overlay_np[stripe_mask, 2] = np.clip(overlay_np[stripe_mask, 2] + noise_b[stripe_mask], 0, 255)
    
    # set semi-transparent alpha (50-150)
    overlay_np[stripe_mask, 3] = np.random.randint(50, 151, size=np.sum(stripe_mask))
    
    # convert back to PIL Image
    overlay = Image.fromarray(overlay_np)
    
    # composite with base image
    base_img = Image.fromarray(img_array).convert('RGBA')
    combined = Image.alpha_composite(base_img, overlay)
    combined = combined.convert('RGB')
    
    # save
    combined.save(f'images2/image_{img_num:05d}.png')